In [24]:
import openai
import requests
from flask import Flask, request, jsonify, render_template
from transformers import pipeline
from amadeus import Client, ResponseError
import pandas as pd
import json

In [37]:
hotels_url = 'https://test.api.amadeus.com/v1/reference-data/locations/hotels/by-city'
params= {
    'cityCode': 'CLE',  # IATA code for Cleveland
    'radius': 5,
    'radiusUnit': 'KM',
    'ratings': '3,4,5',  # Optional: filter by star ratings
    'hotelSource': 'ALL'  # ALL, BEDBANK, or DIRECTCHAIN
}


# For authentication
auth_url = 'https://test.api.amadeus.com/v1/security/oauth2/token'
auth_data = {
    'grant_type': 'client_credentials',
    'client_id': 'RW5x5Bn2aMZYmgyHCyomxPetDteMmM2a',
    'client_secret': 'sKvaa9jS36eO7OCL'
}

# Get access token
auth_response = requests.post(auth_url, data=auth_data)
access_token = auth_response.json()['access_token']

# Use token in API request
headers = {
    'Authorization': f'Bearer {access_token}'
}

response_hotel = requests.get(hotels_url, headers=headers, params=params)
hotel_data = response_hotel.json()

In [33]:
print("Keys in flight_data:", flight_data.keys())

# Try to create a simplified dataframe based on the actual structure
flights_list = []

# Use a safer approach that doesn't assume 'data' exists
if isinstance(flight_data, dict):
    # If flight_data is the top-level dictionary
    if 'data' in flight_data:
        # Process as before
        for offer in flight_data['data']:
            flight_info = {
                'offer_id': offer.get('id', 'ID not available'),
                'price': offer.get('price', {}).get('total', 'Price not available'),
                # Other fields...
            }
            flights_list.append(flight_info)
    # Check for other common response structures
    elif 'flights' in flight_data:
        for flight in flight_data['flights']:
            # Extract information
            flight_info = {
                'flight_id': flight.get('id', 'ID not available'),
                # Add other fields based on structure
            }
            flights_list.append(flight_info)
    else:
        # If none of the expected keys are present, try to extract from the top level
        print("Unexpected structure. Top-level keys:", flight_data.keys())
        # Just take the first few items from the dictionary to see what's available
        sample_data = dict(list(flight_data.items())[:5])
        print("Sample data:", json.dumps(sample_data, indent=2))
elif isinstance(flight_data, list):
    # If flight_data is directly a list
    for item in flight_data:
        flight_info = {
            # Extract fields based on what's available
            # Use item.get() for each field you want
        }
        flights_list.append(flight_info)

# Only create DataFrame if we have data
if flights_list:
    flight_df = pd.DataFrame(flights_list)
    print(flight_df)
else:
    print("No flight data could be extracted")

Keys in flight_data: dict_keys(['errors'])
Unexpected structure. Top-level keys: dict_keys(['errors'])
Sample data: {
  "errors": [
    {
      "status": 400,
      "code": 572,
      "title": "INVALID OPTION",
      "detail": "Invalid query parameter",
      "source": {
        "parameter": "cityCode"
      }
    },
    {
      "status": 400,
      "code": 572,
      "title": "INVALID OPTION",
      "detail": "Invalid query parameter",
      "source": {
        "parameter": "radius"
      }
    },
    {
      "status": 400,
      "code": 572,
      "title": "INVALID OPTION",
      "detail": "Invalid query parameter",
      "source": {
        "parameter": "radiusUnit"
      }
    },
    {
      "status": 400,
      "code": 572,
      "title": "INVALID OPTION",
      "detail": "Invalid query parameter",
      "source": {
        "parameter": "ratings"
      }
    },
    {
      "status": 400,
      "code": 572,
      "title": "INVALID OPTION",
      "detail": "Invalid query parameter",

In [38]:
# Flight search API endpoint
flight_url = 'https://test.api.amadeus.com/v2/shopping/flight-offers'

# Parameters for flight search
params = {
    'originLocationCode': 'CLE',        # Cleveland airport code
    'destinationLocationCode': 'NYC',   # New York airport code (or use JFK, LGA, etc.)
    'departureDate': '2025-05-01',      # Format: YYYY-MM-DD
    'adults': 1,                        # Number of adult passengers
    'max': 10                           # Maximum number of results
}

# Authentication (same as before)
auth_url = 'https://test.api.amadeus.com/v1/security/oauth2/token'
auth_data = {
    'grant_type': 'client_credentials',
    'client_id': 'YOUR_API_KEY',
    'client_secret': 'YOUR_API_SECRET'
}

# Get access token
auth_response = requests.post(auth_url, data=auth_data)
access_token = auth_response.json()['access_token']

# Use token in API request
headers = {
    'Authorization': f'Bearer {access_token}'
}
respone_flight = requests.get(flight_url, headers=headers, params=params)
flight_data = respone_flight.json()

KeyError: 'access_token'

In [35]:
if 'data' in flight_data:
    flights_list = []
    for offer in flight_data['data']:
        # Extract the most important data from each flight offer
        flight_info = {
            'offer_id': offer.get('id', 'ID not available'),
            'price': offer.get('price', {}).get('total', 'Price not available'),
            'currency': offer.get('price', {}).get('currency', 'Currency not available'),
            # Get the first itinerary segment information (typically the outbound flight)
            'departure_airport': offer.get('itineraries', [{}])[0].get('segments', [{}])[0].get('departure', {}).get('iataCode', 'Airport not available'),
            'arrival_airport': offer.get('itineraries', [{}])[0].get('segments', [{}])[-1].get('arrival', {}).get('iataCode', 'Airport not available'),
            'departure_date': offer.get('itineraries', [{}])[0].get('segments', [{}])[0].get('departure', {}).get('at', 'Date not available'),
            'airline': offer.get('itineraries', [{}])[0].get('segments', [{}])[0].get('carrierCode', 'Airline not available')
        }
        flights_list.append(flight_info)
flight_df = pd.DataFrame(flights_list)
print(flights_list)

[]


In [39]:
hotels_list = []
for hotel in hotel_data['data']:
    hotel_info = {
        'name': hotel.get('name', 'Name not available'),
        'hotel_id': hotel.get('hotelId', 'ID not available'),
        'rating': hotel.get('rating', 'Rating not available'),
    }
    hotels_list.append(hotel_info)
df = pd.DataFrame(hotels_list)
print(df)

                            name  hotel_id  rating
0     DBLTREE CLEVELAND DOWNTOWN  DTCLE216       3
1  WYNDHAM PLAYHOUSESQUARE HOTEL  WYCLEPHS       5
2     RADISSON CLEVELAND GATEWAY  RDCLE627       3
3     THE RITZ-CARLTON CLEVELAND  RZCLECLE       5
4           COMFORT INN DOWNTOWN  CICLE167       3
5    HILTON GI CLEVELAND GATEWAY  GICLE246       3
6            UNIVERSITY HOTEL AN  ILCLE101       3
7  INTERCONTINENTAL SUITES HOTEL  ICCLEICB       3
